In [5]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


In [6]:
# #Pyramid
# def batch_pyramid(images):
#     pyramid_list = []
#     for i in tqdm(range(len(images))):
#         img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         G = img.copy()
#         gp = [G]
#         for i in range(4):
#             G = cv2.pyrDown(G)
#             gp.append(G)
#         gp_out = [cv2.resize(gp[2], (128,128)),cv2.resize(gp[3], (128,128))]
#         pyramid_list.append((np.array(gp_out)).astype(np.uint8))
#     pyramid_list = np.array(pyramid_list,dtype = float32).reshape(
#                  [images.shape[0],images.shape[1],images.shape[2],2])/255.0
    
#     print (pyramid_list.shape)
#     return pyramid_list

In [7]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [8]:
# trainImage, trainLabels, testImage, testLabels = load_flower_data()

# trainImage_pyramid = batch_pyramid(trainImage)
# testImage_pyramid = batch_pyramid(testImage)
# nb_classes = 2

# # Convert class vectors to binary class matrices.
# trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
# print (trainLabels)
# testLabels = keras.utils.to_categorical(testLabels, nb_classes)
# print (testLabels)
# print (testLabels.shape)

# np.save('../trainImage.npy', trainImage)
# np.save('../trainLabels.npy', trainLabels)
# np.save('../testImage.npy', testImage)
# np.save('../testLabels.npy', testLabels)
# np.save('../trainImage_pyramid_G.npy', trainImage_pyramid)
# np.save('../testImage_pyramid_G.npy', testImage_pyramid)

# print("[INFO] trainImage matrix: {:.2f}MB".format(
#     (trainImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainLabels matrix: {:.4f}MB".format(
#     (trainLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage matrix: {:.2f}MB".format(
#     (testImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] testLabels matrix: {:.4f}MB".format(
#     (testLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainImage_pyramid matrix: {:.2f}MB".format(
#     (trainImage_pyramid.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage_pyramid matrix: {:.4f}MB".format(
#     (testImage_pyramid.nbytes) / (1024 * 1000.0)))


In [9]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.pyramid_dim = 2  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Pyramid_G_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_pyramid = np.load('../trainImage_pyramid_L.npy')
        self.test_x_pyramid = np.load('../testImage_pyramid_L.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_pyramid, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_pyramid",x_pyramid.get_shape()) # 128, 128, 3 
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_pyramid,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_pyramid = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.pyramid_dim], 
                                name='x_pyramid')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_pyramid = self.train_x_pyramid[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_pyramid= shuffled_set_pyramid[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_pyramid: batch_x_pyramid,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_pyramid: batch_x_pyramid_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramidtest =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [10]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 11.18, total 50 epochs ptime: 585.08
#  [*] Training finished!
#  [*] Best Epoch:  26 , Accuracy:  0.7899999618530273
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Pyramid_G_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Pyramid_G_C5_D1_Kernel(3,3)_128_lrdecay-26
#  [*] Finished testing Best Epoch: 26 , accuracy:  0.7899999618530273 

CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x2x32) [576, bytes: 2304]
cnn/co

Epoch: [ 3] [  19/  40] time: 29.4389, loss: 0.01299319
Epoch: [ 3] [  20/  40] time: 29.6795, loss: 0.02943745
Epoch: [ 3] [  21/  40] time: 29.9211, loss: 0.02637631
Epoch: [ 3] [  22/  40] time: 30.1715, loss: 0.07880777
Epoch: [ 3] [  23/  40] time: 30.4105, loss: 0.03129169
Epoch: [ 3] [  24/  40] time: 30.6518, loss: 0.11158913
Epoch: [ 3] [  25/  40] time: 30.8959, loss: 0.00843893
Epoch: [ 3] [  26/  40] time: 31.1374, loss: 0.08719341
Epoch: [ 3] [  27/  40] time: 31.4027, loss: 0.02692666
Epoch: [ 3] [  28/  40] time: 31.6608, loss: 0.01487016
Epoch: [ 3] [  29/  40] time: 31.9142, loss: 0.07953551
Epoch: [ 3] [  30/  40] time: 32.1662, loss: 0.04378146
Epoch: [ 3] [  31/  40] time: 32.4063, loss: 0.01755796
Epoch: [ 3] [  32/  40] time: 32.6551, loss: 0.05101056
Epoch: [ 3] [  33/  40] time: 32.9089, loss: 0.07818698
Epoch: [ 3] [  34/  40] time: 33.1478, loss: 0.01357483
Epoch: [ 3] [  35/  40] time: 33.3885, loss: 0.05257373
Epoch: [ 3] [  36/  40] time: 33.6377, loss: 0.0

Epoch: [ 7] [   1/  40] time: 69.4242, loss: 0.00918982
Epoch: [ 7] [   2/  40] time: 69.6757, loss: 0.01558356
Epoch: [ 7] [   3/  40] time: 69.9418, loss: 0.00883951
Epoch: [ 7] [   4/  40] time: 70.1971, loss: 0.01180018
Epoch: [ 7] [   5/  40] time: 70.4552, loss: 0.02201157
Epoch: [ 7] [   6/  40] time: 70.7006, loss: 0.00142251
Epoch: [ 7] [   7/  40] time: 70.9537, loss: 0.00411522
Epoch: [ 7] [   8/  40] time: 71.1959, loss: 0.00238882
Epoch: [ 7] [   9/  40] time: 71.4424, loss: 0.00064677
Epoch: [ 7] [  10/  40] time: 71.6918, loss: 0.02313669
Epoch: [ 7] [  11/  40] time: 71.9327, loss: 0.04796354
Epoch: [ 7] [  12/  40] time: 72.1882, loss: 0.00189696
Epoch: [ 7] [  13/  40] time: 72.4298, loss: 0.00230367
Epoch: [ 7] [  14/  40] time: 72.6770, loss: 0.00433115
Epoch: [ 7] [  15/  40] time: 72.9328, loss: 0.00996824
Epoch: [ 7] [  16/  40] time: 73.1763, loss: 0.00440731
Epoch: [ 7] [  17/  40] time: 73.4175, loss: 0.00192828
Epoch: [ 7] [  18/  40] time: 73.6654, loss: 0.0

Epoch: [10] [  24/  40] time: 108.4875, loss: 0.00512200
Epoch: [10] [  25/  40] time: 108.7491, loss: 0.00407509
Epoch: [10] [  26/  40] time: 108.9942, loss: 0.01231092
Epoch: [10] [  27/  40] time: 109.2639, loss: 0.00682776
Epoch: [10] [  28/  40] time: 109.5220, loss: 0.00141083
Epoch: [10] [  29/  40] time: 109.7836, loss: 0.00020878
Epoch: [10] [  30/  40] time: 110.0383, loss: 0.01214281
Epoch: [10] [  31/  40] time: 110.3035, loss: 0.00612296
Epoch: [10] [  32/  40] time: 110.5675, loss: 0.00578333
Epoch: [10] [  33/  40] time: 110.8338, loss: 0.00333702
Epoch: [10] [  34/  40] time: 111.0962, loss: 0.00008041
Epoch: [10] [  35/  40] time: 111.3639, loss: 0.00224566
Epoch: [10] [  36/  40] time: 111.6161, loss: 0.00096051
Epoch: [10] [  37/  40] time: 111.8682, loss: 0.00172923
Epoch: [10] [  38/  40] time: 112.1220, loss: 0.00023669
Epoch: [10] [  39/  40] time: 112.3900, loss: 0.00104710
[10/50] - ptime: 11.0153 loss: 0.01273488 acc: 0.53000 lr: 0.00090000
Epoch: [11] [   0/

Epoch: [14] [   3/  40] time: 149.3055, loss: 0.00118157
Epoch: [14] [   4/  40] time: 149.5738, loss: 0.06316902
Epoch: [14] [   5/  40] time: 149.8421, loss: 0.02641951
Epoch: [14] [   6/  40] time: 150.1083, loss: 0.04221347
Epoch: [14] [   7/  40] time: 150.3837, loss: 0.00231129
Epoch: [14] [   8/  40] time: 150.6486, loss: 0.00160637
Epoch: [14] [   9/  40] time: 150.9180, loss: 0.00705701
Epoch: [14] [  10/  40] time: 151.1861, loss: 0.00318266
Epoch: [14] [  11/  40] time: 151.4540, loss: 0.01780846
Epoch: [14] [  12/  40] time: 151.7215, loss: 0.00751796
Epoch: [14] [  13/  40] time: 151.9939, loss: 0.00011096
Epoch: [14] [  14/  40] time: 152.2584, loss: 0.00216674
Epoch: [14] [  15/  40] time: 152.5267, loss: 0.00711178
Epoch: [14] [  16/  40] time: 152.7954, loss: 0.04370007
Epoch: [14] [  17/  40] time: 153.0632, loss: 0.01504579
Epoch: [14] [  18/  40] time: 153.3106, loss: 0.00053912
Epoch: [14] [  19/  40] time: 153.5673, loss: 0.00050126
Epoch: [14] [  20/  40] time: 1

...

Epoch: [35] [   3/  40] time: 390.0904, loss: 0.00005217
Epoch: [35] [   4/  40] time: 390.3351, loss: 0.00000472
Epoch: [35] [   5/  40] time: 390.6027, loss: 0.00004653
Epoch: [35] [   6/  40] time: 390.8659, loss: 0.00000305
Epoch: [35] [   7/  40] time: 391.1181, loss: 0.00000040
Epoch: [35] [   8/  40] time: 391.3613, loss: 0.00000754
Epoch: [35] [   9/  40] time: 391.6165, loss: 0.00000715
Epoch: [35] [  10/  40] time: 391.8892, loss: 0.00000053
Epoch: [35] [  11/  40] time: 392.1545, loss: 0.00000549
Epoch: [35] [  12/  40] time: 392.4124, loss: 0.00001090
Epoch: [35] [  13/  40] time: 392.6611, loss: 0.00000183
Epoch: [35] [  14/  40] time: 392.9059, loss: 0.00000706
Epoch: [35] [  15/  40] time: 393.1502, loss: 0.00000263
Epoch: [35] [  16/  40] time: 393.3941, loss: 0.00000347
Epoch: [35] [  17/  40] time: 393.6365, loss: 0.00000242
Epoch: [35] [  18/  40] time: 393.8846, loss: 0.00000131
Epoch: [35] [  19/  40] time: 394.1263, loss: 0.00000109
Epoch: [35] [  20/  40] time: 3

Epoch: [38] [  24/  40] time: 429.7095, loss: 0.00001718
Epoch: [38] [  25/  40] time: 429.9771, loss: 0.00000223
Epoch: [38] [  26/  40] time: 430.2455, loss: 0.00000178
Epoch: [38] [  27/  40] time: 430.5063, loss: 0.00000289
Epoch: [38] [  28/  40] time: 430.7642, loss: 0.00000654
Epoch: [38] [  29/  40] time: 431.0181, loss: 0.00000666
Epoch: [38] [  30/  40] time: 431.2618, loss: 0.00000093
Epoch: [38] [  31/  40] time: 431.5021, loss: 0.00000821
Epoch: [38] [  32/  40] time: 431.7605, loss: 0.00000528
Epoch: [38] [  33/  40] time: 432.0136, loss: 0.00003661
Epoch: [38] [  34/  40] time: 432.2602, loss: 0.00002665
Epoch: [38] [  35/  40] time: 432.5216, loss: 0.00000630
Epoch: [38] [  36/  40] time: 432.7783, loss: 0.00000186
Epoch: [38] [  37/  40] time: 433.0425, loss: 0.00002168
Epoch: [38] [  38/  40] time: 433.2951, loss: 0.00006147
Epoch: [38] [  39/  40] time: 433.5531, loss: 0.00000449
[38/50] - ptime: 11.0612 loss: 0.00001559 acc: 0.76000 lr: 0.00072900
Epoch: [39] [   0/

Epoch: [42] [   3/  40] time: 471.5863, loss: 0.00000385
Epoch: [42] [   4/  40] time: 471.8624, loss: 0.00002259
Epoch: [42] [   5/  40] time: 472.1421, loss: 0.00006205
Epoch: [42] [   6/  40] time: 472.4179, loss: 0.00001351
Epoch: [42] [   7/  40] time: 472.6913, loss: 0.00000380
Epoch: [42] [   8/  40] time: 472.9631, loss: 0.00001650
Epoch: [42] [   9/  40] time: 473.2249, loss: 0.00000537
Epoch: [42] [  10/  40] time: 473.4792, loss: 0.00000171
Epoch: [42] [  11/  40] time: 473.7474, loss: 0.00000181
Epoch: [42] [  12/  40] time: 474.0162, loss: 0.00000152
Epoch: [42] [  13/  40] time: 474.2882, loss: 0.00000271
Epoch: [42] [  14/  40] time: 474.5472, loss: 0.00000072
Epoch: [42] [  15/  40] time: 474.8003, loss: 0.00000165
Epoch: [42] [  16/  40] time: 475.0459, loss: 0.00000078
Epoch: [42] [  17/  40] time: 475.3033, loss: 0.00000124
Epoch: [42] [  18/  40] time: 475.5600, loss: 0.00000216
Epoch: [42] [  19/  40] time: 475.8315, loss: 0.00000950
Epoch: [42] [  20/  40] time: 4

Epoch: [45] [  24/  40] time: 514.3768, loss: 0.00000254
Epoch: [45] [  25/  40] time: 514.6516, loss: 0.00000113
Epoch: [45] [  26/  40] time: 514.9079, loss: 0.00000102
Epoch: [45] [  27/  40] time: 515.1675, loss: 0.00000133
Epoch: [45] [  28/  40] time: 515.4318, loss: 0.00000054
Epoch: [45] [  29/  40] time: 515.6925, loss: 0.00000514
Epoch: [45] [  30/  40] time: 515.9621, loss: 0.00002226
Epoch: [45] [  31/  40] time: 516.2117, loss: 0.00000576
Epoch: [45] [  32/  40] time: 516.4758, loss: 0.00000122
Epoch: [45] [  33/  40] time: 516.7351, loss: 0.00000618
Epoch: [45] [  34/  40] time: 516.9977, loss: 0.00001758
Epoch: [45] [  35/  40] time: 517.2597, loss: 0.00004134
Epoch: [45] [  36/  40] time: 517.5177, loss: 0.00001514
Epoch: [45] [  37/  40] time: 517.7827, loss: 0.00000099
Epoch: [45] [  38/  40] time: 518.0392, loss: 0.00000129
Epoch: [45] [  39/  40] time: 518.2994, loss: 0.00000154
[45/50] - ptime: 12.3577 loss: 0.00000695 acc: 0.75000 lr: 0.00065610
Epoch: [46] [   0/

Epoch: [49] [   3/  40] time: 561.3005, loss: 0.00001351
Epoch: [49] [   4/  40] time: 561.5648, loss: 0.00000053
Epoch: [49] [   5/  40] time: 561.8415, loss: 0.00000071
Epoch: [49] [   6/  40] time: 562.1205, loss: 0.00000201
Epoch: [49] [   7/  40] time: 562.4044, loss: 0.00000258
Epoch: [49] [   8/  40] time: 562.6893, loss: 0.00000061
Epoch: [49] [   9/  40] time: 562.9665, loss: 0.00000458
Epoch: [49] [  10/  40] time: 563.2450, loss: 0.00000096
Epoch: [49] [  11/  40] time: 563.5204, loss: 0.00003590
Epoch: [49] [  12/  40] time: 563.8047, loss: 0.00000115
Epoch: [49] [  13/  40] time: 564.0950, loss: 0.00000980
Epoch: [49] [  14/  40] time: 564.3766, loss: 0.00006062
Epoch: [49] [  15/  40] time: 564.6528, loss: 0.00000699
Epoch: [49] [  16/  40] time: 564.9274, loss: 0.00001165
Epoch: [49] [  17/  40] time: 565.2118, loss: 0.00000055
Epoch: [49] [  18/  40] time: 565.5018, loss: 0.00000091
Epoch: [49] [  19/  40] time: 565.7880, loss: 0.00000079
Epoch: [49] [  20/  40] time: 5

In [11]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放